<a href="https://colab.research.google.com/github/tomeliud/Natural-Language-Processings-NLP-on-Swahili-News-Classification-/blob/master/Swahili_News_Classifier_colab_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
!pip install tensorflow

In [16]:
import numpy as np
import pandas as pd
import re
import warnings
warnings.filterwarnings("ignore")
dataset = pd.read_csv('Train.csv') 
dataset

,id,content,category
0,SW0,SERIKALI imesema haitakuwa tayari kuona amani...,Kitaifa
1,SW1,"Mkuu wa Mkoa wa Tabora, Aggrey Mwanri amesiti...",Biashara
2,SW10,SERIKALI imetoa miezi sita kwa taasisi zote z...,Kitaifa
3,SW100,KAMPUNI ya mchezo wa kubahatisha ya M-bet ime...,michezo
4,SW1000,WATANZANIA wamekumbushwa kusherehekea sikukuu...,Kitaifa
...,...,...,...
5146,SW993,RAIS John Magufuli ameendelea kung’ara katika...,Kitaifa
5147,SW994,KAMPENI inayohimiza watafi ti kuandika upya h...,Kitaifa
5148,SW996,"MATUKIO mapya ya malaria kwa kila watu 1,000 ...",Kitaifa
5149,SW997,IMEELEZWA kuwa hakuna sheria yoyote inayokata...,Kitaifa


In [17]:
sentences = dataset['content'].tolist()
sentences 

[' SERIKALI imesema haitakuwa tayari kuona amani na utulivu wa nchi inachezewa huku ikisisitiza uwepo wa umoja kati ya wananchi bila kujali tofauti ya imani, kabila au itikadi yoyote.Hayo yalisemwa na Naibu Waziri wa Mambo ya Ndani ya Nchi, Hamad Yussuf Masauni wakati akifungua semina ya siku mbili iliyofanyika jijini Dar es Salaam ikiwahusisha viongozi wa taasisi za Kiislamu, lengo ikiwa ni kuwakumbusha kuhubiri amani katika sehemu zao.Naibu Waziri amesema mwelekeo na malengo ya Serikali ya Awamu ya Tano ni kukuza maendeleo katika sehemu mbalimbali nchini lengo ikiwa kuinua maisha ya wananchi na nchi kwa ujumla.“Serikali hii imejidhatiti kuhakikisha maendeleo yanakuja kwa kasi na maendeleo hayawezi kuja ikiwa amani na utulivu haupo, sisi kama serikali tutahakikisha tunalinda amani iliyopo ili wananchi wapate kufanya shughuli za kiuchumi bila wasiwasi wowote,” amesema Masauni.Akizungumza wakati wa ufunguzi huo, Shehe wa Mkoa wa Dar es Salaam, Alhaji Alhad Mussa Salum aliihakikishia ser

In [18]:
dataset['content']=dataset['content'].str.lower()
dataset['content']=dataset['content'].str.replace("[^a-zA-Z]"," ") # removes everyting except alphabet (small and caps)

In [26]:
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
word=[]
for i in dataset['content']:
    word.append(word_tokenize(i))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [28]:
filter_sent=[]
stopwords_swahili=["akasema","alikuwa","amesema","alisema","ambayo","ambapo","ambao","baada","basi","bila","cha","chini","hadi","hapo","hata",
                   "hivyo","hiyo","huku","huo","ili","ilikuwa","juu","kama","karibu","katika","kila",
                   "kima","kisha","kubwa","kutoka","kutokana","kwani","kuwa","kwa","kwamba","kwenda","kwenye","la",
                   "lakini","mara","mdogo","mimi","mkubwa","mmoja","moja","muda","mwenye","na",
                   "naye","ndani","ng","ni","nini","nonkungu","pamoja","pia","sana","sasa","sauti",
                   "tafadhali","tena","tu","vile","wa","wakati","wake","walikuwa","wao","watu","wengine",
                   "wote","ya","yake","yangu","yao","yeye","yule","za","zaidi","zake"]
for i in word:
    l=[]
    for j in i:
        if j not in stopwords_swahili:
            l.append(j)
    filter_sent.append(' '.join(l))

In [29]:
dataset['content']=filter_sent

In [30]:
### Create the necessary dummy variables
dataset[['Kitaifa','michezo','Biashara','Burudani']]=pd.get_dummies(dataset['category'])
### Create the necessary dummy variables
dataset.columns
labels=dataset.drop(['id', 'content', 'category'],axis=1)
labels 

ValueError: ignored

In [34]:
#Label encoding  
from sklearn.preprocessing import LabelEncoder
# Categorical boolean mask
categorical_feature_mask = dataset.dtypes==object
# filter categorical columns using mask and turn it into a list
categorical_features = dataset.columns[categorical_feature_mask].tolist()
categorical_features

['id', 'content', 'category']

In [35]:
#getting the length of categorical_features
len(categorical_features)

3

In [36]:
#Getting numerical dummies for the categorical data
columns = ['id', 'content', 'category']
col = dataset[columns]
x = pd.get_dummies(col,drop_first=True)
#checking if the transformation was successful
x

,id_SW1,id_SW10,id_SW100,id_SW1000,id_SW1002,id_SW1003,id_SW1004,id_SW1006,id_SW1007,id_SW1009,id_SW101,id_SW1010,id_SW1011,id_SW1012,id_SW1013,id_SW1015,id_SW1016,id_SW1020,id_SW1021,id_SW1023,id_SW1024,id_SW1025,id_SW1026,id_SW1028,id_SW1029,id_SW103,id_SW1030,id_SW1031,id_SW1032,id_SW1034,id_SW1035,id_SW1036,id_SW1037,id_SW1038,id_SW104,id_SW1042,id_SW1043,id_SW1044,id_SW1045,id_SW1047,...,content_yanga inatarajiwa kuondoka kesho kuifuata usm alger algeria ajili mchezo makundi michuano kombe shirikisho afrika utakaochezwa mjini algiers mwishoni mwa wiki hii akizungumza gazeti hili nsajigwa wanajua wanakwenda kukutana timu ngumu wanajipanga kuhakikisha wanaanza vizuri usm alger miongoni mwa timu afrika tunawaheshimu tunakwenda tukijua wazi hautakuwa mchezo rahisi kikubwa kwetu kujipanga kufanya vizuri timu jangwani inakwenda mechi ikiwa imetoka kupoteza mechi ligi dhidi mtani jadi simba bao mwishoni mwa wiki iliyopita nsajigwa hayo yameshapita wanachoangalia michuano kimataifa kitendo kupoteza mchezo wamejiondoa mbio ubingwa dhidi vinara simba iliyowaacha mbali tofauti pointi kikosi yanga tayari kilianza mazoezi tangu juzi kujiandaa mchezo kesho kitaanza safari shirika ndege emirates kupitia dubai usm alger iliwahi kufika fainali mwaka mchezo ligi mabingwa afrika ilifungwa kwao tp mazembe mchezo unaotazamwa wenye upinzani mkali mchezo yanga itarudi dar es salaam kuwasubiri rayon sport rwanda mei mwaka huu itakwenda ugenini julai mwaka huu kuvaana gor mahia kenya,content_yanga inatarajiwa ugenini nairobi kesho kumenyana gor mahia kenya mechi makundi michuano kombe shirikisho afrika tayari wawakilishi hao tanzania wameshawasili nairobi ajili mechi wakitamba kufanya vizuri licha kuacha nyota kadhaa wanaodaiwa kugoma kutolipwa fedha zao kocha yanga mwinyi zahera aliliambia gazeti hili anatarajia kupata matokeo mazuri mechi licha changamoto zinazowakabili kocha gor mahia dylan kerr amekiri michuano kombe kagame iliyomalizika hivi karibuni dar es salaam kipimo kizuri timu ajili mchezo dhidi yanga yanga haikushiriki michuano kujitoa madai kutokuwa wachezaji wengi hawajalipwa gor mahia ilipata ushindi mwepesi kuifunga timu jeshi kujenga uchumi jku zanzibar mabao kumaliza nafasi tatu michuano iliyofanyika dar es salaam mchezaji mburundi gor mahia ndiye aliyefunga bao kuongoza dakika saba tangu kuanza kipindi pili kabla samuel onyango hajafunga bao ushindi tulipata nafasi tisa kufunga mabao tulifanikiwa kufunga mawili siwezi sema hatujaifanyia kazi tunachotakiwa kuimarika kocha kerr nawapongeza wachezaji wangu wameonesha uwezo mzuri utatusaidia kuelekea mchezo ujao dhidi yanga aliongeza michuano kombe kagame mwaka huu azam fc ilifanikiwa kutetea taji lake kuwafunga mabingwa tanzania bara simba mabao azam waliwatoa gor mahia nusu fainali mashindano hayo mechi mwisho yanga kucheza kundi lake dhidi rayon sports rwanda zilitoka suluhu uwanja taifa dar es salaam wawakilishi hao bara wanacheza gor kesho wakiwa pointi pekee iliyoipata mechi rayon mechi kwanza ilicheza dhidi mc alger algeria kufungwa mabao muhimu yanga kuifunga gor ijiweke mazingira mazuri kusonga mbele kundi d linaloongozwa alger yenye pointi nne ikifuatiwa gor yenye pointi mbili sawa rayon sports yanga ikishika mkia pointi,content_yanga ipo mapambano kuhakikisha inawabakiza nyota waliomaliza mikataba wapinzani simba wamekuwa mstari mbele kuwarubuni nyota hao kujiunga kikosi chao akizungumza gazeti hili mkwasa mikakati iliyopo hivi kumalizika mfungo mwezi mtukufu ramadhani kuwasainisha mikataba mipya wachezaji wamemaliza mikataba inatokana kukamilika mazungumzo nyota hao wiki hii tunatarajia kumalizana thabani kamusoko amisi tambwe wamemaliza mikataba tutakuwa kikao vicent bossou ambaye yupo mipango yetu msimu ujao mkwasa katibu huyo mambo yanakwenda kisomi hawaoni haja kukurupuka vyombo vya habari mitandaoni sababu timu usajili imejipanga vizuri wanaelewa wanachokifanya mkwasa wanataka kujenga kikosi ambacho kitafanya maajabu kufanya vizuri kuvunja rekodi kufika mbali michu

In [37]:
dataset['category'].value_counts()

Kitaifa      2000
michezo      1720
Biashara     1360
Kimataifa      54
Burudani       17
Name: category, dtype: int64

In [38]:
dataset['category'].nunique()

5

In [39]:
dataset.head()

,id,content,category
0,SW0,serikali imesema haitakuwa tayari kuona amani ...,Kitaifa
1,SW1,mkuu mkoa tabora aggrey mwanri amesitisha liki...,Biashara
2,SW10,serikali imetoa miezi sita taasisi zote umma a...,Kitaifa
3,SW100,kampuni mchezo kubahatisha m bet imeingia maku...,michezo
4,SW1000,watanzania wamekumbushwa kusherehekea sikukuu ...,Kitaifa
